In [1]:
import pickle

with open('outfile1', 'rb') as fp:
    baseball_data = pickle.load(fp)

del baseball_data[-1]
del baseball_data[108]
del baseball_data[108]
del baseball_data[108]
del baseball_data[30]
del baseball_data[91]

import json
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
baseball_json = []
for days in baseball_data:
    baseball_json.append(json.loads(days))
for days in baseball_json:
    del days['_comment']
    
joe = json_normalize(baseball_json)
joe = joe.drop(columns=['league.alias', 'league.date',
                 'league.id', 'league.name'])

In [3]:
pd.set_option('display.max_columns', None)
joey = json_normalize(joe.iloc[0,0])
for rows in range(1,180):
    joey = joey.append(json_normalize(joe.iloc[rows, 0]), ignore_index=True, sort=False)

In [4]:
joey.shape

(2445, 141)

In [5]:
joey

,game.attendance,game.away.abbr,game.away.errors,game.away.events,game.away.hits,game.away.id,game.away.loss,game.away.market,game.away.name,game.away.probable_pitcher.era,game.away.probable_pitcher.first_name,game.away.probable_pitcher.id,game.away.probable_pitcher.jersey_number,game.away.probable_pitcher.last_name,game.away.probable_pitcher.loss,game.away.probable_pitcher.preferred_name,game.away.probable_pitcher.win,game.away.runs,game.away.scoring,game.away.starting_pitcher.era,game.away.starting_pitcher.first_name,game.away.starting_pitcher.id,game.away.starting_pitcher.jersey_number,game.away.starting_pitcher.last_name,game.away.starting_pitcher.loss,game.away.starting_pitcher.preferred_name,game.away.starting_pitcher.win,game.away.win,game.away_team,game.broadcast.network,game.coverage,game.day_night,game.double_header,game.duration,game.final.inning,game.final.inning_half,game.game_number,game.home.abbr,game.home.errors,game.home.events,game.home.hits,game.home.id,game.home.loss,game.home.market,game.home.name,game.home.probable_pitcher.first_name,game.home.probable_pitcher.id,game.home.probable_pitcher.jersey_number,game.home.probable_pitcher.last_name,game.home.probable_pitcher.preferred_name,game.home.runs,game.home.scoring,game.home.starting_pitcher.first_name,game.home.starting_pitcher.id,game.home.starting_pitcher.jersey_number,game.home.starting_pitcher.last_name,game.home.starting_pitcher.preferred_name,game.home.win,game.home_team,game.id,game.pitching.blown_save,game.pitching.hold,game.pitching.loss.blown_save,game.pitching.loss.first_name,game.pitching.loss.hold,game.pitching.loss.id,game.pitching.loss.jersey_number,game.pitching.loss.last_name,game.pitching.loss.loss,game.pitching.loss.position,game.pitching.loss.preferred_name,game.pitching.loss.primary_position,game.pitching.loss.save,game.pitching.loss.status,game.pitching.loss.win,game.pitching.save.blown_save,game.pitching.save.first_name,game.pitching.save.hold,game.pitching.save.id,game.pitching.save.jersey_number,game.pitching.save.last_name,game.pitching.save.loss,game.pitching.save.position,game.pitching.save.preferred_name,game.pitching.save.primary_position,game.pitching.save.save,game.pitching.save.status,game.pitching.save.win,game.pitching.win.blown_save,game.pitching.win.first_name,game.pitching.win.hold,game.pitching.win.id,game.pitching.win.jersey_number,game.pitching.win.last_name,game.pitching.win.loss,game.pitching.win.position,game.pitching.win.preferred_name,game.pitching.win.primary_position,game.pitching.win.save,game.pitching.win.status,game.pitching.win.win,game.rescheduled,game.scheduled,game.status,game.venue.address,game.venue.capacity,game.venue.city,game.venue.country,game.venue.field_orientation,game.venue.id,game.venue.location.lat,game.venue.location.lng,game.venue.market,game.venue.name,game.venue.stadium_type,game.venue.state,game.venue.surface,game.venue.zip,game.weather.current_conditions.cloud_cover,game.weather.current_conditions.condition,game.weather.current_conditions.dew_point_f,game.weather.current_conditions.humidity,game.weather.current_conditions.obs_time,game.weather.current_conditions.temp_f,game.weather.current_conditions.wind.direction,game.weather.current_conditions.wind.speed_mph,game.weather.forecast.cloud_cover,game.weather.forecast.condition,game.weather.forecast.dew_point_f,game.weather.forecast.humidity,game.weather.forecast.obs_time,game.weather.forecast.temp_f,game.weather.forecast.wind.direction,game.weather.forecast.wind.speed_mph,game.home.probable_pitcher.era,game.home.probable_pitcher.loss,game.home.probable_pitcher.win,game.home.starting_pitcher.era,game.home.starting_pitcher.loss,game.home.starting_pitcher.win,game.tbd
0,16971.0,CWS,0,[{'pitcher_id': '1019e26a-57a1-4480-aed5-843ba...,15,47f490cd-2f58-4ef7-9dfd-2ad6ba6c1ae8,16,Chicago,White Sox,6.000,Carson,65b0e836-7f64-4ecc-8942-db8a90ba7a98,51,Fulmer,1.0,Carson,1.0,8,"[{'errors': 0, 'hits': 0, 'type': 'inning', 'r...",6.000,Carson,65b0e8